In [16]:
# Load the pre-trained model and tokenizer
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

# Prepare the input text
text = "I'm going to the beach next weekend"
max_length = 64
tokens = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Use the model to predict future date references
outputs = model(**tokens)
predictions = outputs.logits.argmax(-1)
if predictions.sum() == 0:
    print("No labels predicted.")
else:
    labels = [model.config.id2label[label_id] for label_id in predictions[0] if label_id != -100]
    label_map = {i: label for i, label in enumerate(model.config.label2id.keys())}
    labels = [label_map[label_id] for label_id in labels]
    
    # Identify date references using regular expressions
    date_patterns = [
        r"next\s+\b(day|week|month|year)\b",
        r"\d{1,2}(/|-)\d{1,2}(/|-)\d{2,4}",
        r"\d{2,4}(/|-)\d{1,2}(/|-)\d{1,2}",
        r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b\s+\d{1,2}"
    ]
    has_date_reference = False
    for pattern in date_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            start, end = match.span()
            token_ids = tokens["input_ids"][0][start:end]
            ref_labels = [label_map[label_id] for label_id in predictions[0][start:end] if label_id != -100]
            if "B-DAT" in ref_labels or "I-DAT" in ref_labels:
                has_date_reference = True
                break
    
    # Interpret the predictions
    if has_date_reference:
        print("The text contains a future date reference")
    else:
        print("The text does not contain a future date reference")


No labels predicted.
